# es index 优化

## 1. close index

### 测试


* 查看当前各个node segments使用情况

```
curl '172.21.2.40:9200/_cat/nodes?v&h=name,segments.count,segments.memory,segments.index_writer_memory,segments.version_map_memory,segments.fixed_bitset_memory'
name segments.count segments.memory segments.index_writer_memory segments.version_map_memory segments.fixed_bitset_memory
node138 0 0b 0b 0b 0b
node160 13644 304.6mb 0b 0b 0b
node161 13576 299.7mb 0b 0b 0b
node139 0 0b 0b 0b 0b
node159 13711 299.5mb 0b 0b 0b
```

* 执行索引关闭操作

``` 
curl  '172.21.2.40:9200/_cat/indices' -s |grep 2017 |wc -l

100

curl -XPOST '172.21.2.40:9200/*2017*/_close?pretty' -d ''

```


* 查看关闭索引后各个node segments使用情况

```
name segments.count segments.memory segments.index_writer_memory segments.version_map_memory segments.fixed_bitset_memory
node138 0 0b 0b 0b 0b
node160 11517 266.5mb 0b 0b 0b
node161 11534 263.7mb 0b 0b 0b
node139 0 0b 0b 0b 0b
node159 11644 263.1mb 3.9mb 0b 0b
```

* 检查各个node节点io、cpu和内存情况

未发现明显变化 （注意：如果打开索引时，如果数据量较大，cpu和io将有明显升高）




### 生产环境规划

* 先关闭2017年所有index

``` 
curl -XPOST '172.21.0.40:9200/*2017*/_close' -d ''
```

* 编写脚本实现每天定时关闭前120天的index

```
bash -x es_index_daily.sh
```


## 2. force merge index

### 测试

* 查看index 的segments，挑选两个数据量接近的索引

```
curl 172.21.2.40:9200/_cat/indices -s |grep application-2018.04.1 |sort
green open  application-2018.04.10    Z8zP3h7kQkedb0s4-767ZQ 5 1  1304403 0    1.3gb  670.2mb
green open  application-2018.04.11    Ov6zXhDmQq6dtaqbthUCGQ 5 1  2127049 0    1.9gb  988.8mb
green open  application-2018.04.12    Ej1h7bEFTOGM6dQfBxqvBQ 5 1  2542847 0    2.2gb    1.1gb
green open  application-2018.04.13    at_dkhQiSJGkvNFDTUO_mA 5 1  1290054 0    1.2gb  629.1mb
green open  application-2018.04.14    TzRg_GlqT3KPYZpiKXkA8w 5 1   873947 0  927.3mb  463.8mb
green open  application-2018.04.15    eFCkUPwWRdWiaNY9BIFTwQ 5 1   995784 0    983mb  491.2mb
green open  application-2018.04.16    j7-kv7_1SuKS8ZDWB8LH-A 5 1  1221927 0    1.2gb  636.7mb

```

* 查看索引对应的segments数量

```
curl 172.21.2.40:9200/_cat/segments/application-2018.04.16 -s |wc
    127    1778   12222
curl 172.21.2.40:9200/_cat/segments/application-2018.04.13 -s |wc
    113    1582   10769

```

* 对其中一个index做forcemerge

```
curl -XPOST 172.21.2.40:9200/application-2018.04.16/_forcemerge?max_num_segments=1 -d ''

curl 172.21.2.40:9200/_cat/segments/application-2018.04.16 -s |wc
     10     140     970
```

* 对比两个index 查询速度（一次）

```
time curl 172.21.2.40:9200/application-2018.04.16/_search?q=message:lead
real	0m0.021s
user	0m0.008s
sys	0m0.004s
time curl 172.21.2.40:9200/application-2018.04.13/_search?q=message:lead
real	0m1.890s
user	0m0.007s
sys	0m0.000s

```
### 生产环境规划

* 先分批（按月）合并2018年索引

``` 
curl -XPOST '172.21.0.40:9200/application-2018.01.*/_forcemerge?max_num_segments=1‘ -d ''
curl -XPOST '172.21.0.40:9200/application-2018.02.*/_forcemerge?max_num_segments=1‘ -d ''
curl -XPOST '172.21.0.40:9200/application-2018.03.*/_forcemerge?max_num_segments=1‘ -d ''
curl -XPOST '172.21.0.40:9200/application-2018.04.*/_forcemerge?max_num_segments=1‘ -d ''
```

* 编写脚本实现每天定时关闭进行merge前一天index

```
bash -x es_index_daily.sh
```

In [ ]:
!bash -x es_index_daily.sh

+ es_index_log=/tmp/es_index.log
++ date -d '1  days ago' +%Y.%m.%d
usage: date [-jnRu] [-d dst] [-r seconds] [-t west] [-v[+|-]val[ymwdHMS]] ... 
            [-f fmt date | [[[mm]dd]HH]MM[[cc]yy][.ss]] [+format]
+ yesterday=
++ date -d '120  days ago' +%Y.%m.%d
usage: date [-jnRu] [-d dst] [-r seconds] [-t west] [-v[+|-]val[ymwdHMS]] ... 
            [-f fmt date | [[[mm]dd]HH]MM[[cc]yy][.ss]] [+format]
+ daysago=
+ es_url=http://172.21.2.40:9200/
+ index_keywords=("application" "nginx_frontend" "nginx_backend")
++ date +%Y-%m-%d
+ echo 2018-04-17
+ curl -s 'http://172.21.2.40:9200/_cat/nodes?v&h=name,segments.count,segments.memory,segments.index_writer_memory,segments.version_map_memory,segments.fixed_bitset_memory'
